### Modules
A go module is a collection of Go packages.
A Go module therefore looks something like this
```tree
  .
  |-- go.mod
  |-- pkga
  | '-- pkga.go 
  |-- pkgb
  | '-- pkgb.go 
  '-- pkgc
  | '-- pkgc.go
```

`go.mod` specifies the module's dependencies, required minimum go version and other metadata

Modules are at the heart of Go's toolchain used to build and manage Go packages. command such as `go build`, `go test` and `go get` work with modules, but for interacting with modules and managing them, the `go mod` command is used. \
The `go help mod` command will print a list of these commands.

In [ ]:
!go help mod

The `go mod init` command is used to initialize a new module. \
Use `go mod help init` for details

In [ ]:
!go mod help init

Go tooling has support for underlying Version Control Systems(VCS) such as git and mercurial. \
If you plan to host your module on one of these services or an internal VCS, you will most likely want to use a module that can be fetched using the VCS through the `go get` command. You would therefore initialize your module with a name such as `github.com/user/repo`, and someone else could use `go get github.com/user/repo` to fetch that module from the VCS

### Packages
- A collection of Go source files that share the same import path
- These files are almost always in the same directory as each other
- The directory name is almost always the same as the package name.

Consider the tree below
```tree
  .
  |-- go.mod
  |-- smtp
  | |-- smtp.go
  | '-- template.go 
  '-- sound
  | |-- analyzer.go 
  | '-- sound.go
```
- The structure contains 2 directories `smtp` and `sound`, each of which contains `.go` files. Hence, `smtp` and `sound` are considered to be packages
Code within  a package can reference any identifier such as a function, variable  or type within the package, regardless of whether that identifier is exported

**Naming packages**
- The name of the package *must* be decleared at the top of *every* Go source file using the `package` keyword 
- Package names are lowercased, contain only letters and numbers and must start with a letter. Underscores and captital letters are discouraged. 
- Keep package names short and descriptive. use easy to understand abbreviations if necessary.
- Avoid using the name of porpular packages eg 'user', 'conn', 'db'
- Avoid generic or "catch-all" packages such as 'util', 'helpler', 'misc' etc. They are not descriptive, and they are an open invitation for dumping random code.
> Your packages should be focused on the specific functionality and problem space they provide.

**File structure**
- With a few exceptions, the name of the folder and the package are the same. They don't have to match but it is greatly adviced
- All files within the same directory/package have to have to the same package name in the file.
- files names are less strict than package names. The convension is to use lowercase letters and underscores

There really is no best way to organize packages. Keep a few point in mind when organizing packages though;
- Code organization
- Documentation
- Maintainability
- Reusability
- Testing

- Focus: Packages should be singularly focused
- API scope: Export only the most necessary identifiers.
- File organiztion: Packages should be organized in a way that is clear and obvious. Break packages into smaller files having a clear purpose 

**Importing packages and modules**
- Import path for a package is the relative path from the module root to the package diretory. 
- Import path for package or module  always starts with module name.
- The `import` keyword is used along with the fully qualified packag name to import. 
- Multiple import statements can be condensed into a single import statement.
Example:
```go
  import (
    "time"
    "os/exec"
    "github.com/user/repo/sound"
  )
```

It is not uncommon to run into naming collition between two packages when importing. Such collisions will lead to build failures
- We can fix this by using an import alias for one or both packages. For example;
```go
  import (
    "demo/bar"
    pub "demo/foo/bar"
  )
```
`demon/bar` and `demo/foo/bar` all have package name 'bar'. by adding an alias, eg 'pub', before the second import path, we have 'pub' as the alias for 'bar'  in the importing package. \
We can then use identifies from "demo/bar" and "demo/foo/bar" like so; \
```go
  var _ bar.Bar
  var _ pub.Pub // without alias would be 'bar.Pub'
```

### Dependencies
Dependencies are basically 3rd party libraries (package or module) that we can use to get the functionality we need as the standard library will not always be enough.
- To add dependencies to our module, we can use the `go get` command 
- use `go help get` for details on the command.
- Go keeps tract of installed dependencies in the `go.mod` file at the root of the module.

In [ ]:
!go help get

Example working with dependencies; \
Lets import and use the 'github.com/gobuffalo/flect' dependency. This package is used for string manipulation.
- We must first initialize a module (We must work within a module)
- Then we install the dependency
> To see the documentation for any package, use the `go doc` command. Of course you must first get the package with go get if it is not in the standard library. Example: `go doc fmt` to see the docs of fmt package on the terminal

In [ ]:
// initialize module 'notebook'
// note that we are using shebang here because we are running shell commands from jupyter notebook cells
!go mod init notebook

In [ ]:
// get required package. 
!go get github.com/gobuffalo/flect

In [ ]:
package main

import (
  "fmt"
  "github.com/gobuffalo/flect"
)

func main() {
  fmt.Println(flect.Pluralize("Cat"))
	fmt.Println(flect.Dasherize("Hello World"))
}

//run program with command `go run main.go`

**The go.sum file**
- When adding a third-party dependency to the module, the go toolchain will generate and manage a `go.sum` file.
- The `go.sum` file contains a list of all the dependencies, direct and indirect, that are required by the module.
- It also contains the hash fo the source code and `go.mod` file for each dependency.
- The `go.sum` file should be checked into the version control. 
- The `go.sum` acts both as a security measure to  prevent malicious code from being added to the module and as a source of truth for the dependencies that are required by the module.
> The `go.sum` file is manageed by the Go toolchain and shouuld not be manually modified

**Updating Dependencies**
- Use the `go get` command with the `-u` flag to update a package. Exmaple: 
```bash
  go get -u github.com/gobuffalo/flect
```
- `go.mod` and `go.sum` will be auto updated to include the new dependency, it's version and any other dependencies it requires.
- to update all direct dependencies of our module, we an use the `go get -u` command without specifying package name.

Go modules are versioned using [semantic versioning](https://semver.org) \
Example: `v1.2.3`
- `1` is the major version of release
- `2` is the minor version of release
- `3` is the patch version of release

Go modules also include a concept known as [semantic import versioning.](https://research.swtch.com/vgo-import)
- If the latest version of the package is `v1.2.3`, the import path should be `github.com/user/repo`
- If the package is updated to `v2.0.0`, then we need to change the import path to reflect the new version as shown below;
```go
  package foo
  import "github.com/user/repo/v2"
```
- We are therefore able to have multiple versions of a package in our module and use aliases to allow use use multiple versions of the same package side by sid; Example;
```go
  package main
  import (
    "fmt"
    "log"

    one "github.com/gofrs/uuid"
    three "github.com/gofrs/uuid/v4"
  )

  func main(){
    id1 := one.NewV4()
    fmt.Printf("UUID v4: %s\n", id1)

    id2 := three.New()
    fmt.Printf("UUID v4: %s\n", id2)
  }
```
the `go.mod` file will show the specific version for each module being used in our application.

**Cyclical Imports**
- Its unacceptable for packages to import themselves (package 'foo' cannot import package 'bar' while package 'bar' is also importing package 'foo'). This will lead to a compilation error.